### Import Libraries

In [1]:
import pandas as pd

### Load Dataset

In [2]:
CSV_FILENAME = "ths-st3 compiled dataset.csv" #<-- update csv name
df = pd.read_csv(CSV_FILENAME)
emin_list = ['e_face_yf', 'e_bbox_yf', 'e_lbbox_yf', 'e_face_mp', 'e_bbox_mp', 'e_lbbox_mp', 'e_face_yn', 'e_bbox_yn', 'e_lbbox_yn']

### Train Test Split

In [3]:
from sklearn.model_selection import train_test_split
random_state = 100
train, test = train_test_split(df, test_size = 0.2, random_state=random_state)

### Performance Metrics

In [4]:
def positive_error(actual, pred):
    total = 0
    count = 0
    a = pred
    for b in actual:
        if a > b:
            total += a - b
            count += 1
    if count == 0: count += 1
    return total / count

def concealment_ratio(actual, pred):
    count = 0
    a = pred
    for b in actual:
        if a >= b:
            count+= 1
            
    return count / len(actual)

### Naive emin w/ Intervals from MIN to MAX

In [5]:
def split_range(min_value, max_value, num_intervals):
    interval = (max_value - min_value) / num_intervals
    intervals = (min_value + i * interval for i in range(num_intervals))
    return intervals

In [6]:
result_list = []

for emin in emin_list:
    df_describe = df[emin].describe()
    intervals = split_range(df_describe['min'], df_describe['max'], 10) 
    for interval in intervals:
        row = {}
        row['emin_perturbation_type'] = emin
        row['emin_value'] = interval
        row['concealment_ratio'] = concealment_ratio(test[emin], interval)
        row['positive_error'] = positive_error(test[emin], interval)
        result_list.append(row)

result_df = pd.DataFrame(result_list)

In [7]:
result_df[result_df['emin_perturbation_type'] == 'e_face_yf']

,emin_perturbation_type,emin_value,concealment_ratio,positive_error
0,e_face_yf,0.005,0.000604,0.000000
1,e_face_yf,0.098,0.038276,0.027026
2,e_face_yf,0.191,0.398066,0.043019
3,e_face_yf,0.284,0.832595,0.091923
4,e_face_yf,0.377,0.965149,0.167955
5,e_face_yf,0.470,0.989726,0.255871
6,e_face_yf,0.563,0.994762,0.347356
7,e_face_yf,0.656,0.996978,0.439515
8,e_face_yf,0.749,0.998590,0.531718
9,e_face_yf,0.842,0.999597,0.624143


### Naive emin w/ MIN, 25%, 50%, 75%, MAX

In [8]:
indices = ['min', '25%', '50%', '75%', 'max']
result_list = []

for emin in emin_list:
    df_describe = df[emin].describe()
    for index in indices:
        row = {}
        row['emin_perturbation_type'] = emin
        row['emin_value_type'] = index
        row['emin_value'] = df_describe[index]
        row['concealment_ratio'] = concealment_ratio(test[emin], df_describe[index])
        row['positive_error'] = positive_error(test[emin], df_describe[index])
        result_list.append(row)

result_df = pd.DataFrame(result_list)

In [9]:
result_df[result_df['emin_perturbation_type'] == 'e_face_yf']

,emin_perturbation_type,emin_value_type,emin_value,concealment_ratio,positive_error
0,e_face_yf,min,0.005,0.000604,0.000000
1,e_face_yf,25%,0.165,0.269944,0.039571
2,e_face_yf,50%,0.210,0.516317,0.051754
3,e_face_yf,75%,0.260,0.767728,0.076278
4,e_face_yf,max,0.935,1.000000,0.717015


### Naive emin w/ Same Intervals

In [10]:
def floating_range(start, stop, step):
    value = start
    while value < stop:
        yield value
        value += step

In [11]:
result_list = []

for emin in emin_list:
    df_describe = df[emin].describe()
    for interval in floating_range(0, 1.05, 0.05):
        row = {}
        row['emin_perturbation_type'] = emin
        row['emin_value'] = interval
        row['concealment_ratio'] = concealment_ratio(test[emin], interval)
        row['positive_error'] = positive_error(test[emin], interval)
        result_list.append(row)

result_df = pd.DataFrame(result_list)

In [12]:
result_df[result_df['emin_perturbation_type'] == 'e_face_yf']

,emin_perturbation_type,emin_value,concealment_ratio,positive_error
0,e_face_yf,0.00,0.000000,0.000000
1,e_face_yf,0.05,0.007051,0.021406
2,e_face_yf,0.10,0.043715,0.029026
3,e_face_yf,0.15,0.189766,0.030621
4,e_face_yf,0.20,0.457091,0.045524
5,e_face_yf,0.25,0.717566,0.069740
6,e_face_yf,0.30,0.882353,0.103275
7,e_face_yf,0.35,0.951652,0.144069
8,e_face_yf,0.40,0.973811,0.189370
9,e_face_yf,0.45,0.986906,0.236572
